# 3. Feature Engineering and data preparation
* Now we have performed EDA and done preliminary cleaning of the data we can now start preparing the data ready for usage by our chosen algorithm
         
* Going to restructure data to be supervised and create some features which may be useful in aiding model to pick up the underlying relationships
   
   
* The onehot and binary encoded features didn't end up getting used.
    * They were required for RNN based models but they proved too slow for this complex problem
    * So I ended up using LGBM: which can handle categorical values without having to encode them and encoding binary/onehot actually worsens their performance so I just used the ordinal encoding instead
# Imports


In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime

# pd.display optioons

In [70]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# loading data

In [71]:
df_energy = pd.read_pickle("../Data/Preprocessed_Data/consumption_clustered.pkl")
df_energy

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7410,5.48250,7.224,4.951,4.19650,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,5.1075,4.818,3.931,4.2170,4.503,4.8160,5.129,5.395,1
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.4690,24.09950,24.730,24.597,22.32650,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,14.6315,14.936,16.174,20.3960,24.618,19.8925,15.167,11.751,0
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0265,16.22475,12.423,18.379,15.85025,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,7.3520,7.384,14.425,16.0650,17.705,13.3355,8.966,4.633,5
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.3150,21.49350,19.672,21.023,20.35450,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,11.7255,10.974,19.646,21.8195,23.993,19.9170,15.841,14.452,5
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.3880,41.99750,48.607,54.760,48.73150,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,21.9445,8.351,9.957,17.9140,25.871,36.0725,46.274,16.901,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,2.3040,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.9760,10.30550,10.635,10.650,10.39950,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,9.2235,9.216,9.336,9.6840,10.032,9.8945,9.757,9.480,3
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,10.3010,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,5.8620,6.81600,7.770,5.091,5.60000,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,3.8230,4.818,3.357,8.1315,12.906,8.0140,3.122,3.401,3
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,6.8355,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,6.449,...,7.2410,6.46800,5.695,8.143,7.56800,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,5.9980,6.077,7.761,6.6080,5.455,5.5670,5.679,8.148,3
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,24.6880,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,28.7260,29.28500,29.844,31.899,30.96900,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,20.1925,20.406,23.668,27.7900,31.912,29.1125,26.313,24.201,0


In [72]:
df_addInfo = pd.read_pickle("../Data/Preprocessed_Data/addInfo_knn.pkl")
df_addInfo

,meter_id,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0x02b7418b821b2edeeb2baa23ba1580566b4b0e43,3.0,0.0,0.0,1.0,0.0
1,0xaf32b2b497e9eb9c3f3de907aedc6a7914a11f41,2.0,0.0,0.0,0.0,1.0
2,0xdd45ab28a7a40eadc04607d6da42e5ad923b1339,3.0,0.0,0.0,1.0,0.0
3,0x2b538a5ebfc3780aef25227267063cb6410d943b,4.0,1.0,0.0,0.0,0.0
4,0xee032ed498c9e99a4c65e0f39896fb30e513c6b7,3.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
3243,0xdf6a54b92985398af03423426e744c6d40ee8c69,4.0,0.0,0.0,1.0,0.0
3244,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,3.0,1.0,0.0,0.0,0.0
3245,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,3.0,0.0,0.0,1.0,0.0
3246,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,3.0,0.0,0.0,1.0,0.0


In [73]:
# joining based on meter_id to assign labels to the original dataset (with all meter readings)
df_energy_addInfo = pd.merge(df_energy, df_addInfo, on="meter_id",how="inner")
df_energy_addInfo

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,5.1075,4.818,3.931,4.2170,4.503,4.8160,5.129,5.395,1,2.0,0.0,0.0,0.0,1.0
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,14.6315,14.936,16.174,20.3960,24.618,19.8925,15.167,11.751,0,4.0,1.0,0.0,0.0,0.0
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,7.3520,7.384,14.425,16.0650,17.705,13.3355,8.966,4.633,5,3.0,0.0,0.0,1.0,0.0
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,11.7255,10.974,19.646,21.8195,23.993,19.9170,15.841,14.452,5,3.0,0.0,0.0,1.0,0.0
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,21.9445,8.351,9.957,17.9140,25.871,36.0725,46.274,16.901,4,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,2.3040,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,9.2235,9.216,9.336,9.6840,10.032,9.8945,9.757,9.480,3,3.0,0.0,0.0,1.0,0.0
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,10.3010,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,3.8230,4.818,3.357,8.1315,12.906,8.0140,3.122,3.401,3,2.0,0.0,0.0,0.0,1.0
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,6.8355,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,6.449,...,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,5.9980,6.077,7.761,6.6080,5.455,5.5670,5.679,8.148,3,2.0,0.0,0.0,0.0,1.0
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,24.6880,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,20.1925,20.406,23.668,27.7900,31.912,29.1125,26.313,24.201,0,3.0,1.0,0.0,0.0,0.0


# Training set 
## Restructuring for the KNN model

### Converting to a supervised learning problem

* Transforming data from a time series structure into a supervised learning structure
    * restrecturing dataframe with 9 columns, 8 inputs, 1 output (meter_reading)
    
* Currently making single step predictions past on last 7 steps
    * hence will need to iteratively feed the output into itself 365 times to get entire years worth of prediction
    
* TODO : In hyper parameter tuning will want to experiment with different values for n_steps, but this is a good place to start
    * May also want to experiment with different number of outputs too

In [74]:
def to_supervised(n_steps, df_original):

    #defining the new structure for the dataframe using the defining number of n_steps used to make our prediction
    n_prev_meters = []
    for i in range(n_steps,0,-1):
        n_prev_meters.append("energy n-"+str(i))
    df_energy_train = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced"]+n_prev_meters+["meter_reading"])

    #iteratively populating the training set
    date = pd.date_range(datetime(2017, 1, 1+n_steps), datetime(2017, 12, 31))#outside of loop as doesn't need to be recaculated

    #for each row in the dataframe
    for row_id in tqdm(range(len(df_original))):

        #getting relevant values for this row
        row = df_original.iloc[row_id]

        ### adding the various meta_data ###
        meter_id = row.meter_id
        energy_cluster = row.labels
        num_bedrooms = row.num_bedrooms
        detached = row.dwelling_type_detached_house
        flat = row.dwelling_type_flat
        semi_detached = row.dwelling_type_semi_detached_house
        terraced = row.dwelling_type_terraced_house

        #formatting this row into dataframe with timeseries patterns over rows instead of columns
        df_meter = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced"])

        df_meter["date"]=date #needs to be first as it determines length of columns
        df_meter["meter_id"]=meter_id
        df_meter["energy_cluster"]=energy_cluster
        df_meter["num_bedrooms"]=num_bedrooms
        df_meter["detached"]=detached
        df_meter["flat"]=flat
        df_meter["semi_detached"]=semi_detached
        df_meter["terraced"]=terraced






        ### adding the previous n_steps outputs as inputs ###

        prev_readings, this_reading = [[]],[]
        #iterating through all the dates for this meter (skipping meter_id column up to len row - 6 metadata columns - n_steps which overflow)
        for first_input_index in range(1, len(row)-6-n_steps):
            cell = row[first_input_index]

            #find end of this sequence
            output_index = first_input_index + n_steps
            #check if we are beyond the sequence (if our output index > the last date in the year)
            if output_index > len(row)-6-1:
                break

            #getting the input and output sequences then appending them
            inputs, outputs = row[first_input_index:output_index], row[output_index]
            this_reading.append(outputs)
            prev_readings.append(inputs)

        #converting to dataframe
        df_prev_readings=pd.DataFrame(data=prev_readings, columns=n_prev_meters).iloc[1:].reset_index(drop=True)#removing first row as it is empty list
        df_this_readings=pd.DataFrame(data=this_reading, columns=["meter_reading"])
        df_readings=pd.concat([df_prev_readings,df_this_readings],axis=1)

        #adding the readings and meta data into one dataframe
        df_meter = pd.concat([df_meter,df_readings],axis=1)

        #finished converting this row so adding it to the dataframe
        df_energy_train = pd.concat([df_energy_train, df_meter],axis=0).reset_index(drop=True)

    return df_energy_train

In [75]:
#number of previous steps we are using in our predictions
df_energy_train=to_supervised(7, df_energy_addInfo)
df_energy_train

  0%|          | 0/3248 [00:00<?, ?it/s]

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-08,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-09,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-10,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-11,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-12,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162779,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,0,3.0,1.0,0.0,0.0,0.0,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585
1162780,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,0,3.0,1.0,0.0,0.0,0.0,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110
1162781,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,0,3.0,1.0,0.0,0.0,0.0,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040
1162782,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,0,3.0,1.0,0.0,0.0,0.0,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970


### Removing the nan entries

In [76]:
#checking the number of unique meter_id's in this
df_energy_train["meter_id"].nunique()

3248

In [77]:
# removing rows containing nans as the model won't be able to use them
df_energy_train_no_nan = df_energy_train.dropna(axis=0).reset_index(drop=True)
df_energy_train_no_nan

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-10,1,2.0,0.0,0.0,0.0,1.0,3.5170,2.9330,3.0850,3.2370,4.4710,3.7410,5.4825,7.2240
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-11,1,2.0,0.0,0.0,0.0,1.0,2.9330,3.0850,3.2370,4.4710,3.7410,5.4825,7.2240,4.9510
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-12,1,2.0,0.0,0.0,0.0,1.0,3.0850,3.2370,4.4710,3.7410,5.4825,7.2240,4.9510,4.1965
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-13,1,2.0,0.0,0.0,0.0,1.0,3.2370,4.4710,3.7410,5.4825,7.2240,4.9510,4.1965,3.4420
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-14,1,2.0,0.0,0.0,0.0,1.0,4.4710,3.7410,5.4825,7.2240,4.9510,4.1965,3.4420,3.5090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617628,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,0,3.0,1.0,0.0,0.0,0.0,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585
617629,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,0,3.0,1.0,0.0,0.0,0.0,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110
617630,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,0,3.0,1.0,0.0,0.0,0.0,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040
617631,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,0,3.0,1.0,0.0,0.0,0.0,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970


In [78]:
#checking the number of unique meter_id's in this
df_energy_train_no_nan["meter_id"].nunique()

3236

In [79]:
#require the last days row as a minimum in order to iteratively make our predictions
## so we want this to be the same as the number of meter_id's
len(df_energy_train_no_nan[df_energy_train_no_nan.date == '2017-12-31'])

3236

In [80]:
#what id's are missing
missing_ids = np.setdiff1d(df_energy_train["meter_id"],df_energy_train_no_nan["meter_id"],)
missing_ids

array(['0x240e5e22734a44a174b7dabcf1ea00d70d9ec168',
       '0x2ce3d582a1316db5bcfe405cbd6070268944778e',
       '0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471',
       '0x391e6c2169c27de797ccbdf2d623365da28a6d3d',
       '0x423fa805ddb0cba9bdb4460f9a78540287eefd0e',
       '0x795fa14a925f90f11752454da39988048db6e042',
       '0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b',
       '0x81fa8eddb2b09393d3719984ca5520cb50f45efd',
       '0xab6f16ec971923ef1acd4693705b28bcfe7a7115',
       '0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11',
       '0xe2154c841409bba5b83d208c83c33938d1a02474',
       '0xe41c230484378b2312fbf1fc99d289e0b51a3c52'], dtype=object)

In [81]:
#getting a dataframe of these missing id's from the original dataset

df_missing = pd.DataFrame()

for each in missing_ids:
    df_missing = pd.concat([df_missing, df_energy_addInfo[df_energy_addInfo.meter_id == each]],axis=0).reset_index(drop=True)
    
df_missing

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4015,0.426,0.426,0.426,0.426,0.426,0.426,1,3.0,0.0,0.0,1.0,0.0
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.7530,9.367,8.490,7.613,8.079,8.545,9.714,1,3.0,0.0,0.0,1.0,0.0
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
5,0x795fa14a925f90f11752454da39988048db6e042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
6,0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
7,0x81fa8eddb2b09393d3719984ca5520cb50f45efd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
8,0xab6f16ec971923ef1acd4693705b28bcfe7a7115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0
9,0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,0.0,1.0,0.0


# Dealing with the meters that don't exist (cluster 5)
* Cluster 5 is the one which contains all the missing values
    * So will treat cluster 5 different from the other clusters

* We can see that the reason they do not exist is as they do go not have a sequence of non missing values >= n_steps
    * Solution could be don't sent entire day to missing if it contains for these entries and just use the known 
        * From below however can see that isn't reasonable as they contain so many missing values their end 
        
* Most reasonabe solution then will likely be to just use the average values of the known days 

## Trying just ignoring nans and aggregating

In [82]:
# loading the simple aggregated days
df_ignored = pd.read_pickle("../Data/Preprocessed_Data/consumption_daily_nan_ignored.pkl").replace(0,np.nan)

In [83]:
#joining based on meter id to get the extra columns
df_missing_ignored = pd.merge(df_ignored, df_missing[{"meter_id","labels","num_bedrooms","dwelling_type_detached_house","dwelling_type_flat","dwelling_type_semi_detached_house","dwelling_type_terraced_house"}], on="meter_id", how="inner")
df_missing_ignored

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,dwelling_type_flat,dwelling_type_terraced_house,labels,dwelling_type_semi_detached_house,num_bedrooms,dwelling_type_detached_house
0,0x81fa8eddb2b09393d3719984ca5520cb50f45efd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.435,0.480,0.541,0.456,0.663,0.431,0.385,0.183,0.072,0.085,0.168,NaN,0.180,0.270,NaN,0.516,NaN,0.446,0.265,0.0,0.0,1,1.0,3.0,0.0
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.126,0.127,0.126,0.127,0.128,0.127,0.127,0.128,0.127,0.127,2.485,NaN,9.753,9.367,NaN,7.613,NaN,8.545,9.714,0.0,0.0,1,1.0,3.0,0.0
3,0x795fa14a925f90f11752454da39988048db6e042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.376,0.693,0.568,0.501,0.893,0.373,0.739,0.578,0.176,0.103,0.768,NaN,0.258,0.202,NaN,0.423,NaN,2.842,0.245,0.0,0.0,1,1.0,3.0,0.0
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
5,0xe41c230484378b2312fbf1fc99d289e0b51a3c52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
6,0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
7,0xab6f16ec971923ef1acd4693705b28bcfe7a7115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.296,6.069,1.354,NaN,NaN,0.638,2.216,1.566,NaN,0.057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.705,0.0,0.0,1,1.0,3.0,0.0
8,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.083,0.083,0.084,0.082,0.081,0.082,0.083,0.084,0.083,0.084,0.084,NaN,0.084,0.084,NaN,0.082,NaN,0.082,0.084,0.0,0.0,1,1.0,3.0,0.0
9,0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.330,14.140,14.073,14.387,14.597,14.932,13.050,13.163,13.120,12.935,12.513,NaN,12.750,12.406,NaN,13.572,NaN,13.438,12.649,0.0,0.0,1,1.0,3.0,0.0


In [84]:
# interpolating those missing values
df_interpolating_temp = df_missing_ignored.drop(["meter_id","labels","num_bedrooms","dwelling_type_detached_house","dwelling_type_flat","dwelling_type_semi_detached_house","dwelling_type_terraced_house"], axis=1)
#interpolating those days that are still missing limiting to 1
df_interpolating_temp=df_interpolating_temp.interpolate(method="linear", axis=1, limit = 1) 
#combining dataframe
df_interpolating_temp=pd.concat([df_missing_ignored.meter_id, df_interpolating_temp],axis=1)
df_missing_ignored=pd.concat([df_interpolating_temp, df_missing_ignored[{"labels","num_bedrooms","dwelling_type_detached_house","dwelling_type_flat","dwelling_type_semi_detached_house","dwelling_type_terraced_house"}]],axis=1)

df_missing_ignored

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,dwelling_type_flat,dwelling_type_terraced_house,labels,dwelling_type_semi_detached_house,num_bedrooms,dwelling_type_detached_house
0,0x81fa8eddb2b09393d3719984ca5520cb50f45efd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.435,0.480,0.541,0.456000,0.663,0.431,0.385000,0.183,0.0720,0.085,0.168000,0.1740,0.180,0.270,0.3930,0.516,0.4810,0.446,0.265,0.0,0.0,1,1.0,3.0,0.0
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.126,0.127,0.126,0.127000,0.128,0.127,0.127000,0.128,0.1270,0.127,2.485000,6.1190,9.753,9.367,8.4900,7.613,8.0790,8.545,9.714,0.0,0.0,1,1.0,3.0,0.0
3,0x795fa14a925f90f11752454da39988048db6e042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.376,0.693,0.568,0.501000,0.893,0.373,0.739000,0.578,0.1760,0.103,0.768000,0.5130,0.258,0.202,0.3125,0.423,1.6325,2.842,0.245,0.0,0.0,1,1.0,3.0,0.0
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
5,0xe41c230484378b2312fbf1fc99d289e0b51a3c52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
6,0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1.0,3.0,0.0
7,0xab6f16ec971923ef1acd4693705b28bcfe7a7115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.296,6.069,1.354,1.115333,NaN,0.638,2.216000,1.566,0.8115,0.057,0.906778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.705,0.0,0.0,1,1.0,3.0,0.0
8,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.083,0.083,0.084,0.082000,0.081,0.082,0.083000,0.084,0.0830,0.084,0.084000,0.0840,0.084,0.084,0.0830,0.082,0.0820,0.082,0.084,0.0,0.0,1,1.0,3.0,0.0
9,0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.330,14.140,14.073,14.387000,14.597,14.932,13.050000,13.163,13.1200,12.935,12.513000,12.6315,12.750,12.406,12.9890,13.572,13.5050,13.438,12.649,0.0,0.0,1,1.0,3.0,0.0


## Using average values for those that are missing

In [85]:
# just going to use the average for those days which only have nans
df_missing_avg=df_missing.copy()
df_missing_avg["2017-12-31"]=df_energy["2017-12-31"].mean()
df_missing_avg["2017-12-30"]=df_energy["2017-12-30"].mean()
df_missing_avg["2017-12-29"]=df_energy["2017-12-29"].mean()
df_missing_avg["2017-12-28"]=df_energy["2017-12-28"].mean()
df_missing_avg["2017-12-27"]=df_energy["2017-12-27"].mean()
df_missing_avg["2017-12-26"]=df_energy["2017-12-26"].mean()
df_missing_avg["2017-12-25"]=df_energy["2017-12-25"].mean()
df_missing_avg["2017-12-24"]=df_energy["2017-12-24"].mean()

df_missing_avg

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
5,0x795fa14a925f90f11752454da39988048db6e042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
6,0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
7,0x81fa8eddb2b09393d3719984ca5520cb50f45efd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
8,0xab6f16ec971923ef1acd4693705b28bcfe7a7115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0
9,0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495,1,3.0,0.0,0.0,1.0,0.0


## Now rerunning the to_supervised with this new dataset and combining them

In [86]:
#turning the missing values where the average has been imputed, into a supervised learning problem
df_supervised_missing = to_supervised(7,df_missing_avg).dropna(axis=0).reset_index(drop=True)
#concatenating this with the df_energy_train
df_energy_train_with_missing = pd.concat([df_supervised_missing, df_energy_train_no_nan],axis=0).reset_index(drop=True)

df_energy_train_with_missing

  0%|          | 0/12 [00:00<?, ?it/s]

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,2017-12-31,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,2017-12-31,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,2017-12-31,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,2017-12-31,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,2017-12-31,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,0,3.0,1.0,0.0,0.0,0.0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,0,3.0,1.0,0.0,0.0,0.0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,0,3.0,1.0,0.0,0.0,0.0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,0,3.0,1.0,0.0,0.0,0.0,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000,17.397000


## Feature engineering 
#### Dummy variables for the date
* Cyclically encoded
    * As if ordinally encoded it will imply that January (0) and December (12) are far from each other 
    


* My thinking was
    * Cyclically encode day of year as 1st Jan is close to 31st Dec
    * Cyclically encode month for the same reason
    * Not cyclically encoded day of week or month as they exist within the month and there is an ordinal relationship within that month
    * Extra dummy variable for specifying if it's a weekend or not such that it can more easily facilitate learning it


* Will experiment with different combinations of these in the RNN to see which performs the best
    * Eg does the dummy variable for saying it's a weekend or not improve performance or is it better to just use day_of_week?
        * etc.

In [87]:
#defining the new structure with the cyclically encoded date columns
df_energy_train_with_dummies = pd.DataFrame(columns=['meter_id', 'date', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week', 'day_of_month', 'month_ord', 'month_sin', 'month_cos', 'is_weekend','energy_cluster', 'num_bedrooms', 'detached',
       'flat', 'semi_detached', 'terraced', 'energy n-7', 'energy n-6',
       'energy n-5', 'energy n-4', 'energy n-3', 'energy n-2', 'energy n-1',
       'meter_reading'], data=df_energy_train_with_missing)

#cyclically encoding day of year
day_of_year = df_energy_train_with_dummies["date"].dt.dayofyear.astype(int)
df_energy_train_with_dummies["day_of_year_sin"] = np.sin(2 * np.pi * day_of_year/364.0)
df_energy_train_with_dummies["day_of_year_cos"] = np.cos(2 * np.pi * day_of_year/364.0)

#cyclically encoding month
month = df_energy_train_with_dummies["date"].dt.month.astype(int)
df_energy_train_with_dummies["month_ord"] = month
df_energy_train_with_dummies["month_sin"] = np.sin(2 * np.pi * month/11.0)
df_energy_train_with_dummies["month_cos"] = np.cos(2 * np.pi * month/11.0)

#day of week not cyclically encoded as it exists within month
df_energy_train_with_dummies["day_of_week"]  = df_energy_train_with_dummies["date"].dt.dayofweek.astype(int)

#day of month not cyclically encoded as it exists within month
df_energy_train_with_dummies["day_of_month"]  = df_energy_train_with_dummies["date"].dt.day.astype(int)

#dummy variable for is_weekend
df_energy_train_with_dummies["is_weekend"]=0
df_energy_train_with_dummies.loc[df_energy_train_with_dummies.day_of_week>4, "is_weekend"] = 1

# displaying the dataframe
df_energy_train_with_dummies.tail(10)

,meter_id,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
617635,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-22,-1.376535e-01,0.990480,4,22,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,28.4120,40.2510,30.0840,26.0580,27.8730,21.9990,26.2320,14.0360
617636,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-23,-1.205367e-01,0.992709,5,23,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,40.2510,30.0840,26.0580,27.8730,21.9990,26.2320,14.0360,15.9900
617637,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-24,-1.033839e-01,0.994642,6,24,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,30.0840,26.0580,27.8730,21.9990,26.2320,14.0360,15.9900,15.5505
617638,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-25,-8.620038e-02,0.996278,0,25,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,26.0580,27.8730,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110
617639,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-26,-6.899114e-02,0.997617,1,26,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,27.8730,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970
617644,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970,15.2370


# encoding the ID
* Encoding via binary encoding
* One hot encoding has a linear growth rate of n whereas binary encoding has a growth rate of log_2(n)
* Our n is 3500 so:
    * One hot encoding would add 3500 extra features
    * Binary encoding would add log_2(3500) extra features = 12

* Hence, allows us to uniquely describe the different id's with far fewer features and hence resulting in a much less sparse search space

## encoding for the whole dataset

In [88]:
import pandas as pd
import category_encoders as ce

binary_encoder = ce.BinaryEncoder(cols=['meter_id'])
df_energy_train_final = pd.concat([df_energy_train_with_dummies["meter_id"],binary_encoder.fit_transform(df_energy_train_with_dummies)],axis=1)

if(os.path.exists("../Data/Prepared_Data/Supervised/preapared_consumption_all.pkl")==False):
        df_energy_train_final.to_pickle("../Data/Prepared_Data/Supervised/consumption_all.pkl")

C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [89]:
df_energy_train_final

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,0,0,1,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,0,0,1,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,0,0,1,0,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,0,0,1,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000,17.397000


## encoding for each cluster

In [90]:
from IPython.display import display

# splitting by energy cluster
df_train_clusters = {k: v for (k, v) in df_energy_train_with_dummies.groupby('energy_cluster')}

#iterating through each cluster
for i in range(len(df_train_clusters)):
    
    #binary encoding the meter_id
    binary_encoder = ce.BinaryEncoder(cols=['meter_id'])
    df_train_clusters[i] = pd.concat([df_train_clusters[i]["meter_id"],binary_encoder.fit_transform(df_train_clusters[i])],axis=1)

    #dropping the no longer needed "energy cluster" column
    df_train_clusters[i] = df_train_clusters[i].drop("energy_cluster",axis=1)
    
    print(f"cluster {i} lenth: {len(df_train_clusters[i])}")
    
    # saving these clusters to pickle
    df_train_clusters[i].to_pickle(f"../Data/Prepared_Data/Supervised/consumption_cluster_{i}.pkl")
    
    display(df_train_clusters[i])

C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


cluster 0 lenth: 82483


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
34,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,4.0,1.0,0.0,0.0,0.0,18.0570,18.3450,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300
35,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,18.3450,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970
36,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265
37,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265,20.0560
38,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265,20.0560,22.5460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,1.0,0.0,0.0,0.0,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970


C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


cluster 1 lenth: 158584


...

C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
100,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,4.0,1.0,0.0,0.0,0.0,30.378,27.224,29.9250,32.6260,30.5070,35.3880,41.9975,48.6070
101,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,27.224,29.925,32.6260,30.5070,35.3880,41.9975,48.6070,54.7600
102,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,29.925,32.626,30.5070,35.3880,41.9975,48.6070,54.7600,48.7315
103,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,32.626,30.507,35.3880,41.9975,48.6070,54.7600,48.7315,42.7030
104,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,30.507,35.388,41.9975,48.6070,54.7600,48.7315,42.7030,35.0450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615492,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,28.918,22.920,22.4260,25.3950,25.6550,25.9150,33.9030,40.9595
615493,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,22.920,22.426,25.3950,25.6550,25.9150,33.9030,40.9595,48.0160
615494,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,22.426,25.395,25.6550,25.9150,33.9030,40.9595,48.0160,46.2830
615495,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,1.0,0.0,0.0,0.0,25.395,25.655,25.9150,33.9030,40.9595,48.0160,46.2830,44.5500


C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


cluster 5 lenth: 142890


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
56,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,16.32400,8.21138,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300
57,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,8.21138,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900
58,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025
59,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025,13.32150
60,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025,13.32150,13.82100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615850,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.69300,11.70600,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850
615851,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.70600,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000
615852,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000,13.87550
615853,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000,13.87550,12.44100


# Prediction set 
* making an empty prediction set we will populate
* done using same strategy as above

In [91]:
### Creating a prediction set in the same structure as df_energy_addInfo so we can reuse the same code to format it ###

# making the prediction set
df_preds = pd.DataFrame(columns=["meter_id"])
df_preds["meter_id"]=df_energy_addInfo.meter_id

#adding the date columns
for i in tqdm(pd.date_range(datetime(2018,1,1),datetime(2018,12,31),freq="D"),position=0):
    df_preds[str(i.date())]=np.nan
    
#getting the missing columns
cols_to_use = df_energy_addInfo.columns.difference(df_preds.columns)[365:] #after the 365 date columns
cols_to_use = np.concatenate((cols_to_use,["meter_id"]))

df_preds = pd.merge(df_preds, df_energy_addInfo[cols_to_use], on="meter_id",how="inner")
df_preds

  0%|          | 0/365 [00:00<?, ?it/s]

,meter_id,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21,2018-01-22,2018-01-23,2018-01-24,...,2018-12-13,2018-12-14,2018-12-15,2018-12-16,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house,labels,num_bedrooms
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1,2.0
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,4.0
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,5,3.0
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,5,3.0
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,3,3.0
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3,2.0
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3,2.0
3246,0x47218b46abb2fcaade487a211911406dc6e13730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,3.0


In [92]:
def pred_to_supervised(n_steps, df_original):

    #defining the new structure for the dataframe using the defining number of n_steps used to make our prediction
    n_prev_meters = []
    for i in range(n_steps,0,-1):
        n_prev_meters.append("energy n-"+str(i))
    df_energy_pred = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced"]+n_prev_meters+["meter_reading"])

    #iteratively populating the training set
    date = pd.date_range(datetime(2018, 1, 1), datetime(2018, 12, 31))#outside of loop as doesn't need to be recaculated

    #for each row in the dataframe
    for row_id in tqdm(range(len(df_original))):

        #getting relevant values for this row
        row = df_original.iloc[row_id]

        ### adding the various meta_data ###
        meter_id = row.meter_id
        energy_cluster = row.labels
        num_bedrooms = row.num_bedrooms
        detached = row.dwelling_type_detached_house
        flat = row.dwelling_type_flat
        semi_detached = row.dwelling_type_semi_detached_house
        terraced = row.dwelling_type_terraced_house

        #formatting this row into dataframe with timeseries patterns over rows instead of columns
        df_meter = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced"])

        df_meter["date"]=date #needs to be first as it determines length of columns
        df_meter["meter_id"]=meter_id
        df_meter["energy_cluster"]=energy_cluster
        df_meter["num_bedrooms"]=num_bedrooms
        df_meter["detached"]=detached
        df_meter["flat"]=flat
        df_meter["semi_detached"]=semi_detached
        df_meter["terraced"]=terraced

        #finished converting this row so adding it to the dataframe
        df_energy_pred = pd.concat([df_energy_pred, df_meter],axis=0).reset_index(drop=True)

    return df_energy_pred

In [93]:
### Turning into a supervised problem ###

#number of previous steps we are using in our predictions
df_preds_super=pred_to_supervised(7, df_preds)
df_preds_super

  0%|          | 0/3248 [00:00<?, ?it/s]

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-01,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-02,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-03,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-04,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-05,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-27,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-28,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-29,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-30,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
### doing the date based dummies ###

#defining the new structure with the cyclically encoded date columns
df_preds_dummies = pd.DataFrame(columns=['meter_id', 'date', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week', 'day_of_month', 'month_ord','month_sin', 'month_cos', 'is_weekend','energy_cluster', 'num_bedrooms', 'detached',
       'flat', 'semi_detached', 'terraced', 'energy n-7', 'energy n-6',
       'energy n-5', 'energy n-4', 'energy n-3', 'energy n-2', 'energy n-1',
       'meter_reading'], data=df_preds_super)

#cyclically encoding day of year
day_of_year = df_preds_dummies["date"].dt.dayofyear.astype(int)
df_preds_dummies["day_of_year_sin"] = np.sin(2 * np.pi * day_of_year/364.0)
df_preds_dummies["day_of_year_cos"] = np.cos(2 * np.pi * day_of_year/364.0)

#cyclically encoding month
month = df_preds_dummies["date"].dt.month.astype(int)
df_preds_dummies["month_ord"] = month
df_preds_dummies["month_sin"] = np.sin(2 * np.pi * month/11.0)
df_preds_dummies["month_cos"] = np.cos(2 * np.pi * month/11.0)

#day of week not cyclically encoded as it exists within month
df_preds_dummies["day_of_week"]  = df_preds_dummies["date"].dt.dayofweek.astype(int)

#day of month not cyclically encoded as it exists within month
df_preds_dummies["day_of_month"]  = df_preds_dummies["date"].dt.day.astype(int)

#dummy variable for is_weekend
df_preds_dummies["is_weekend"]=0
df_preds_dummies.loc[df_preds_dummies.day_of_week>4, "is_weekend"] = 1

# displaying the dataframe
df_preds_dummies

,meter_id,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-01,1.726064e-02,0.999851,0,1,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-02,3.451614e-02,0.999404,1,2,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-03,5.176135e-02,0.998659,2,3,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-04,6.899114e-02,0.997617,3,4,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-05,8.620038e-02,0.996278,4,5,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-27,-5.176135e-02,0.998659,3,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-28,-3.451614e-02,0.999404,4,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-29,-1.726064e-02,0.999851,5,29,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-30,-2.449294e-16,1.000000,6,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# encoding the ID

## encoding for the whole dataset

In [95]:
df_preds_dummies

,meter_id,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-01,1.726064e-02,0.999851,0,1,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-02,3.451614e-02,0.999404,1,2,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-03,5.176135e-02,0.998659,2,3,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-04,6.899114e-02,0.997617,3,4,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2018-01-05,8.620038e-02,0.996278,4,5,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-27,-5.176135e-02,0.998659,3,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-28,-3.451614e-02,0.999404,4,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-29,-1.726064e-02,0.999851,5,29,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2018-12-30,-2.449294e-16,1.000000,6,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# joining based on meter_id to getting the missing binary_encoded meter_id's
cols_to_use = df_energy_train_final.columns.difference(df_preds_dummies.columns)
cols_to_use = np.concatenate((cols_to_use,["meter_id"]))
df_preds_temp = pd.merge(df_preds_dummies, df_energy_train_final.drop_duplicates("meter_id")[cols_to_use], on="meter_id",how="inner")

#restructuring columns to be in expected order
df_preds_final=pd.DataFrame(columns=df_energy_train_final.columns)
for column_name in df_preds_final.columns:
    df_preds_final[column_name]=df_preds_temp[column_name]

#saving this to pickle
df_preds_final.to_pickle("../Data/Prepared_Data/Supervised/consumption_all_preds.pkl")

df_preds_final

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,1,1,0,1,2018-01-01,1.726064e-02,0.999851,0,1,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,1,1,0,1,2018-01-02,3.451614e-02,0.999404,1,2,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,1,1,0,1,2018-01-03,5.176135e-02,0.998659,2,3,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,1,1,0,1,2018-01-04,6.899114e-02,0.997617,3,4,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,1,1,0,1,2018-01-05,8.620038e-02,0.996278,4,5,1,0.540641,0.841254,0,1,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2018-12-27,-5.176135e-02,0.998659,3,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2018-12-28,-3.451614e-02,0.999404,4,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2018-12-29,-1.726064e-02,0.999851,5,29,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2018-12-30,-2.449294e-16,1.000000,6,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## encoding for each cluster

In [97]:
from IPython.display import display

# splitting by energy cluster
df_preds_clusters = {k: v for (k, v) in df_preds_dummies.groupby('energy_cluster')}

#iterating through each cluster
for i in range(len(df_preds_clusters)):
    
    #dropping the no longer needed "energy cluster" column
    df_preds_clusters[i] = df_preds_clusters[i].drop("energy_cluster",axis=1)
    
    #joining based on meter_id to getting the missing binary_encoded meter_id's
    cols_to_use = df_preds_clusters[i].columns.difference(df_train_clusters[i].columns)
    cols_to_use = np.concatenate((cols_to_use,["meter_id"]))
    df_preds_temp = pd.merge(df_preds_clusters[i], df_train_clusters[i].drop_duplicates("meter_id")[cols_to_use], on="meter_id",how="inner")

    #restructuring columns to be in expected order
    df_preds_clusters[i]=pd.DataFrame(columns= df_train_cslusters[i].columns)
    for column_name in df_preds_clusters[i].columns:
        df_preds_clusters[i][column_name]=df_train_clusters[i][column_name]
    
    print(f"cluster {i} lenth: {len(df_preds_clusters[i])}")
    
    # saving these clusters to pickle
    df_preds_clusters[i].to_pickle(f"../Data/Prepared_Data/Supervised/consumption_cluster_{i}_preds.pkl")
    
    display(df_preds_clusters[i])

cluster 0 lenth: 82483


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
34,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,4.0,1.0,0.0,0.0,0.0,18.0570,18.3450,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300
35,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,18.3450,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970
36,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,17.4190,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265
37,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,16.4930,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265,20.0560
38,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,17.2510,23.4690,24.0995,24.7300,24.5970,22.3265,20.0560,22.5460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,21.9990,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,26.2320,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,14.0360,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,1,1,0,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,1.0,0.0,0.0,0.0,15.9900,15.5505,15.1110,16.5060,18.7585,21.0110,19.2040,17.3970


cluster 1 lenth: 158584


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,1,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,1,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,1,0,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,1,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613702,0x13a47388be5dee69085241e22535f96882602521,0,1,1,0,1,0,0,0,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,1.878000,1.876000,1.911000,1.881000,1.877000,1.873000,1.939000,2.753500
613703,0x13a47388be5dee69085241e22535f96882602521,0,1,1,0,1,0,0,0,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,1.876000,1.911000,1.881000,1.877000,1.873000,1.939000,2.753500,3.568000
613704,0x13a47388be5dee69085241e22535f96882602521,0,1,1,0,1,0,0,0,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,1.911000,1.881000,1.877000,1.873000,1.939000,2.753500,3.568000,2.696000
613705,0x13a47388be5dee69085241e22535f96882602521,0,1,1,0,1,0,0,0,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,1.881000,1.877000,1.873000,1.939000,2.753500,3.568000,2.696000,1.824000


cluster 2 lenth: 6428


...

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
188,0xaf8627889fff7ba6fa91bfae6b05636453d032f1,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,3.0,1.0,0.0,0.0,0.0,5.840,6.519,6.9180,7.3170,10.5800,11.5230,13.7335,15.9440
189,0xaf8627889fff7ba6fa91bfae6b05636453d032f1,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,6.519,6.918,7.3170,10.5800,11.5230,13.7335,15.9440,6.2740
190,0xaf8627889fff7ba6fa91bfae6b05636453d032f1,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,6.918,7.317,10.5800,11.5230,13.7335,15.9440,6.2740,8.0345
191,0xaf8627889fff7ba6fa91bfae6b05636453d032f1,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,7.317,10.580,11.5230,13.7335,15.9440,6.2740,8.0345,9.7950
192,0xaf8627889fff7ba6fa91bfae6b05636453d032f1,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,10.580,11.523,13.7335,15.9440,6.2740,8.0345,9.7950,12.1590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616924,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0,1,0,0,0,0,0,0,1,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,2.0,0.0,0.0,0.0,1.0,5.259,5.258,6.7670,5.9190,5.9980,6.0770,7.7610,6.6080
616925,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0,1,0,0,0,0,0,0,1,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,2.0,0.0,0.0,0.0,1.0,5.258,6.767,5.9190,5.9980,6.0770,7.7610,6.6080,5.4550
616926,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0,1,0,0,0,0,0,0,1,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,2.0,0.0,0.0,0.0,1.0,6.767,5.919,5.9980,6.0770,7.7610,6.6080,5.4550,5.5670
616927,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0,1,0,0,0,0,0,0,1,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,2.0,0.0,0.0,0.0,1.0,5.919,5.998,6.0770,7.7610,6.6080,5.4550,5.5670,5.6790


cluster 4 lenth: 27449


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
100,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,4.0,1.0,0.0,0.0,0.0,30.378,27.224,29.9250,32.6260,30.5070,35.3880,41.9975,48.6070
101,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,27.224,29.925,32.6260,30.5070,35.3880,41.9975,48.6070,54.7600
102,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,29.925,32.626,30.5070,35.3880,41.9975,48.6070,54.7600,48.7315
103,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,32.626,30.507,35.3880,41.9975,48.6070,54.7600,48.7315,42.7030
104,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,4.0,1.0,0.0,0.0,0.0,30.507,35.388,41.9975,48.6070,54.7600,48.7315,42.7030,35.0450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615492,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,28.918,22.920,22.4260,25.3950,25.6550,25.9150,33.9030,40.9595
615493,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,22.920,22.426,25.3950,25.6550,25.9150,33.9030,40.9595,48.0160
615494,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,1.0,0.0,0.0,0.0,22.426,25.395,25.6550,25.9150,33.9030,40.9595,48.0160,46.2830
615495,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,0,1,0,0,1,0,1,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,1.0,0.0,0.0,0.0,25.395,25.655,25.9150,33.9030,40.9595,48.0160,46.2830,44.5500


cluster 5 lenth: 142890


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
56,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-10,-3.384131e-01,0.940998,6,10,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,16.32400,8.21138,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300
57,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-11,-3.221204e-01,0.946699,0,11,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,8.21138,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900
58,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-12,-3.057318e-01,0.952118,1,12,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,9.81869,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025
59,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-13,-2.892521e-01,0.957253,2,13,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.42600,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025,13.32150
60,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0,0,0,0,0,0,0,0,0,0,1,2017-12-14,-2.726862e-01,0.962103,3,14,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,6.39850,20.02650,16.22475,12.42300,18.37900,15.85025,13.32150,13.82100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615850,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.69300,11.70600,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850
615851,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,11.70600,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000
615852,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,3.0,0.0,0.0,1.0,0.0,9.78300,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000,13.87550
615853,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,0,1,0,1,1,1,1,0,1,1,1,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,3.0,0.0,0.0,1.0,0.0,13.41500,17.32450,21.23400,15.70700,15.50850,15.31000,13.87550,12.44100
